In [123]:
# Python script to read stock info from csv file

import pandas as pd;
import os;
import psycopg2;

In [124]:
# Connect to RDS

# Retrieve RDS log in info from text file
file = open('../private.txt', 'r');
lines = file.readlines();

for i,l in enumerate(lines):
    if l[-1] == '\n':
        lines[i] = l[0:-1];
        
database_t = lines[2];
user_t = lines[3];
password_t = lines[4];
host_t = lines[5];
port_t = lines[6];

# Try Connection
try:
    db = psycopg2.connect(
        database= database_t,
        user= user_t,
        password= password_t,
        host= host_t,
        port= port_t
    )
except:
    print("ERROR: Unable to connect to the RDS server.")
    

cur = db.cursor()
cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")

for table in cur.fetchall():
    print(table)


('users',)
('investments',)
('stocks',)
('transactions',)
('prices',)


In [125]:
# Read CSV files

path = '../data/stock_market_dataset/stocks'

files = os.listdir(path)

# Skip first two files which have already been inserted into DB
files = files[2:]

# TICKER.csv
#     0 Date
#     1 Open
#     2 High
#     3 Low
#     4 Close
#     5 Adj Close
#     6 Volume


for f in files:
    fpath = '../data/stock_market_dataset/stocks/'+ f
    data = pd.read_csv(fpath)
    
    ticker = f[:-4]
    squery = """INSERT INTO stocks (ticker, stock_name) VALUES ('%s', 'name needed');"""%ticker;
    cur.execute(squery);
    db.commit();
    
    for row in data.itertuples(index=False):
        date = row[0]
        opn = str(row[1])
        high = str(row[2])
        low = str(row[3])
        close = str(row[4])
        adj = str(row[5])
        vol = str(row[6])
            
        pquery = """INSERT INTO prices (ticker, date, open, high, low, close, adj_close, volume) 
        VALUES ('%s', '%s', %s, %s, %s, %s, %s, %s);"""%(ticker, date, opn, high, low, close, adj, vol);
        
        cur.execute(pquery);
        db.commit();
    
    print(f, ticker, data)



ANTE.csv ANTE             Date        Open        High        Low       Close   Adj Close  \
0     2007-11-07   97.500000  113.550003  96.000000  104.500000  104.500000   
1     2007-11-08   97.500000  101.750000  92.500000   95.000000   95.000000   
2     2007-11-09   92.500000   98.650002  91.000000   97.000000   97.000000   
3     2007-11-12   90.000000   96.500000  90.000000   92.800003   92.800003   
4     2007-11-13   91.550003   94.400002  90.000000   90.000000   90.000000   
5     2007-11-14   95.000000   99.250000  91.050003   95.500000   95.500000   
6     2007-11-15   94.000000   95.400002  92.500000   92.500000   92.500000   
7     2007-11-16   98.599998   98.599998  89.150002   89.949997   89.949997   
8     2007-11-19   88.849998   91.349998  86.400002   90.849998   90.849998   
9     2007-11-20   82.050003   89.849998  81.750000   86.250000   86.250000   
10    2007-11-21   82.500000   82.550003  78.000000   81.250000   81.250000   
11    2007-11-23   81.250000   92.0999

KeyboardInterrupt: 